# Section 4：PythonでAPIにリクエストを送ろう

## requestsモジュールでリクエストを送信する

In [ ]:
import requests

In [ ]:
# エンドポイント
url = 'http://webservice.recruit.co.jp/hotpepper/gourmet/v1/'

In [ ]:
params = {
    'key': 'YOUR API KEY',
    'keyword': '東京駅',
    'format': 'json'
}

In [ ]:
res = requests.get(url, params)

In [ ]:
res

In [ ]:
res.json()

In [ ]:
res.json()['shop']

In [ ]:
res.json()

In [ ]:
res.json()['results']['shop']

In [ ]:
shops = res.json()['results']['shop']
# 先頭3行だけ表示
shops[:3]

In [ ]:
# 店舗数の確認
len(shops)

# Section 5：もっとたくさんのデータを取得しよう

## 10件以上のデータを取得するには

### 1〜100件目のデータを取得

In [ ]:
start = 1
count = 100
params = {
    'key': 'YOUR API KEY',
    'keyword': '東京駅',
    'start': start, # 追加
    'count': count, # 追加
    'format': 'json'
}
res = requests.get(url, params)
shops = res.json()['results']['shop']
shops[:3]

In [ ]:
len(shops)

In [ ]:
res.json()['results']['results_returned']

### 101件目以降のデータを取得

In [ ]:
start = 101 # 101件目から取得
count = 100 # 最大取得件数なのでこのままでもOK
params = {
    'key': 'YOUR API KEY',
    'keyword': '東京駅',
    'start': start,
    'count': count,
    'format': 'json'
}
res2 = requests.get(url, params)
res2.json()

In [ ]:
shops2 = res2.json()['results']['shop']
# 取得件数
len(shops2)

In [ ]:
res2.json()['results']['results_start']

### 2つのリストを結合する

In [ ]:
shops += shops2

In [ ]:
len(shops)

# Section 6：APIで取得したデータを加工しよう

## 店舗のデータから必要な情報のみを取り出す

In [ ]:
shop = shops[0]

In [ ]:
datum = {
    'name': shop['name'],
    'address': shop['address'],
    'capacity': shop['capacity'],
    'access': shop['access'],
    'urls': shop['urls']['pc'],
    'open': shop['open'],
    'budget': shop['budget']['name']
}
datum

## 全店舗のデータから必要な情報のみを取り出す

In [ ]:
# 新しいデータを格納するリストを空で定義
data = []
# 1店舗ずつ取り出し、shopに代入
for shop in shops:
    datum = {
        'name': shop['name'],
        'address': shop['address'],
        'capacity': shop['capacity'],
        'access': shop['access'],
        'urls': shop['urls']['pc'],
        'open': shop['open'],
        'budget': shop['budget']['name']
    }
    # 必要な情報のみを取り出したdatumをdataに追加    
    data.append(datum)

In [ ]:
# 先頭5件
data[:5]

In [ ]:
len(data)

# Section 7：pandasのDataFrame型に変換して分析しよう

## リストのデータをpandasに読み込ませる

In [ ]:
import pandas as pd
# リスト -> DataFrame
df = pd.DataFrame(data)

In [ ]:
df.head()

## 文字列から必要な情報のみを抽出

In [ ]:
_budget = df['budget'][0]
_budget

## 下限の金額を取り出す

In [ ]:
_budget.split('～')

In [ ]:
int(_budget.split('～')[0])

## 上限の金額を取り出す

In [ ]:
_budget.split('～')[1]

In [ ]:
_budget.split('～')[1].replace('円', '')

In [ ]:
int(_budget.split('～')[1].replace('円', ''))

## まとめ

In [ ]:
# 空のリストを定義
lower_budgets = []
upper_budgets = []
# budgetカラムにある各値を順に_budgetに代入
for _budget in df['budget']:
    # _budgetに空文字が入っている場合を避けるために、if文を入れる
    if _budget != '':
        lower_budget = int(_budget.split('～')[0])
        upper_budget = int(_budget.split('～')[1].replace('円', ''))
    else:
        lower_budget = 0
        upper_budget = 0
            
    lower_budgets.append(lower_budget)
    upper_budgets.append(upper_budget)

In [ ]:
# 予算下限
df['lower_budget'] = lower_budgets
# 予算上限
df['upper_budget'] = upper_budgets

In [ ]:
df.head()

# Section 8：条件を指定して必要なデータを抽出する

In [ ]:
df['upper_budget'] <= 4000

In [ ]:
df['upper_budget'][157]

In [ ]:
df_filtered = df[df['upper_budget'] <= 4000]
df_filtered.head()

In [ ]:
df_filtered.shape